In [1]:
!ls ./datasets/kaggle/

MMasseyOrdinals.csv		MNCAATourneySeeds.csv		  MTeams.csv
MNCAATourneyCompactResults.csv	MRegularSeasonCompactResults.csv


In [2]:
!ls ./datasets/torvik/mens/

team_2018.csv  team_2022.csv  top30_wins_2018.csv  top30_wins_2022.csv
team_2019.csv  team_2023.csv  top30_wins_2019.csv  top30_wins_2023.csv
team_2021.csv  team_2024.csv  top30_wins_2021.csv  top30_wins_2024.csv


In [3]:
import pandas as pd
import numpy as np

In [4]:
m_teams = pd.read_csv("./datasets/kaggle/MTeams.csv")
m_teams.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2025
1,1102,Air Force,1985,2025
2,1103,Akron,1985,2025
3,1104,Alabama,1985,2025
4,1105,Alabama A&M,2000,2025


In [5]:
teamIDToName = {}
teamNameToID = {}

for row in range(m_teams.shape[0]):
    teamID = m_teams.iloc[row]["TeamID"].item()
    teamName = m_teams.iloc[row]["TeamName"]
    teamIDToName[teamID] = teamName
    teamNameToID[teamName] = teamID

In [6]:
m_massey_ordinals = pd.read_csv("./datasets/kaggle/MMasseyOrdinals.csv")
m_massey_ordinals[m_massey_ordinals["SystemName"] == "POM"].tail()

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
5520900,2025,121,POM,1476,318
5520901,2025,121,POM,1477,330
5520902,2025,121,POM,1478,353
5520903,2025,121,POM,1479,342
5520904,2025,121,POM,1480,350


In [7]:
m_ncaa_regular_season = pd.read_csv("./datasets/kaggle/MRegularSeasonCompactResults.csv")
m_ncaa_regular_season[m_ncaa_regular_season["Season"] == 2024].head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
181682,2024,0,1101,64,1329,59,A,0
181683,2024,0,1103,81,1355,75,A,0
181684,2024,0,1104,105,1287,73,H,0
181685,2024,0,1112,122,1288,59,H,0
181686,2024,0,1114,71,1402,66,H,0


In [8]:
m_ncaa_tourney_compact = pd.read_csv("./datasets/kaggle/MNCAATourneyCompactResults.csv")
m_ncaa_tourney_compact[m_ncaa_tourney_compact["Season"] == 2024].head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
2451,2024,134,1161,67,1438,42,N,0
2452,2024,134,1447,71,1224,68,N,0
2453,2024,135,1160,60,1129,53,N,0
2454,2024,135,1212,88,1286,81,N,0
2455,2024,136,1112,85,1253,65,N,0


In [9]:
m_ncaa_tourney_seeds = pd.read_csv("./datasets/kaggle/MNCAATourneySeeds.csv")
m_ncaa_tourney_seeds[m_ncaa_tourney_seeds["Season"] == 2024].head()

,Season,Seed,TeamID
2490,2024,W01,1163
2491,2024,W02,1235
2492,2024,W03,1228
2493,2024,W04,1120
2494,2024,W05,1361


In [19]:
# Iterate through all Torvik team names, and check if it exists in the Kaggle Teams dataset.
# If no match exists, find the match and populate the conversion map.
# After some thinking, I realize it's more efficient to check the names of teams in the tournament, not all D1 schools.

# torvikToKaggleTeamName = {
#     "Arkansas Pine Bluff": "Ark Pine Bluff",
#     "Central Michigan": "C Michigan",
#     "FIU": "Florida Intl",
#     "Illinois Chicago": "IL Chicago",
#     "Florida Atlantic": "FL Atlantic",
#     "Cal St. Northridge": "CS Northridge",
#     "Abilene Christian": "Abilene Chr",
#     "UMKC": "Missouri KC",
#     "Northern Illinois": "N Illinois",
#     "Kent St": "Kent",
#     "Central Arkansas": "Cent Arkansas",
#     "American": "American Univ",
#     "UTSA": "UT San Antonio",
#     "Prairie View A&M": "Prairie View",
#     "Georgia Southern": "Ga Southern",
#     "Tennessee Martin": "TN Martin",
#     "LIU": "LIU Brooklyn",
#     "Loyola Chicago": "Loyola-Chicago",
#     "Maryland Eastern Shore": "MD E Shore",
#     "Fairleigh Dickinson": "F Dickinson",
#     "Queens": "Queens NC",
#     "Nebraska Omaha": "NE Omaha",
# }

# Go through all Torvik team names, make sure we can match them with the team names in the Kaggle dataset
kaggleToTorvikTeamNames = {
    "FL Atlantic": "Florida Atlantic",
    "St Mary's CA": "Saint Mary's",
    "Col Charleston": "Charleston",
    "S Dakota St": "South Dakota St.",
    "Grambling": "Grambling St.",
    "NC State": "N.C. State",
    "WKU": "Western Kentucky",
    "F Dickinson": "Fairleigh Dickinson",
    "TAM C. Christi": "Texas A&M Corpus Chris",
    "Kent": "Kent St.",
    "Kennesaw": "Kennesaw St.",
    "CS Fullerton": "Cal St. Fullerton",
    "TX Southern": "Texas Southern",
    "Loyola-Chicago": "Loyola Chicago",
    "SF Austin": "Stephen F. Austin",
    "N Dakota St": "North Dakota St.",
    "Abilene Chr": "Abilene Christian",
}

def preprocess_torvik_dataframe(year, torvik_df, kaggle_teamNames_round_64):
    torvik_teams = torvik_df["TeamName"]
    matches_torvik_teamName = lambda kaggle_teamName: ((torvik_teams == kaggle_teamName).sum().item() == 1)
    
    def mutate_torvik_teamName(old, new):
        torvik_df.loc[torvik_df["TeamName"] == old, "TeamName"] = new
        assert torvik_df.loc[torvik_df["TeamName"] == new].shape[0] == 1
    
    for kaggle_teamName in kaggle_teamNames_round_64:
        if matches_torvik_teamName(kaggle_teamName):
            continue
            
        if kaggle_teamName.startswith("St "):
            test_teamNames = [
                "Saint" + kaggle_teamName[kaggle_teamName.index("St") + len("St"):],
                "St." + kaggle_teamName[kaggle_teamName.index("St") + len("St"):],
            ]
            # print(f"No match for '{kaggle_teamName}'\t checking if '{test_teamName}' exists")
            matched = False
            for test_teamName in test_teamNames:
                if matches_torvik_teamName(test_teamName):
                    print(f"Mutating Torvik dataframe team name from '{test_teamName}' -> '{kaggle_teamName}'")
                    mutate_torvik_teamName(test_teamName, kaggle_teamName)
                    matched = True
                    break
                    
            if matched:
                continue
                
        if kaggle_teamName.endswith("St"):
            # Looks like Kaggle uses St, while Torvik uses St.
            # We'll just remove the period at the end
            test_teamName = kaggle_teamName + "."
            # print(f"No match for '{kaggle_teamName}'\t checking if '{test_teamName}' exists")
            if matches_torvik_teamName(test_teamName):
                print(f"Mutating Torvik dataframe team name from '{test_teamName}' -> '{kaggle_teamName}'")
                mutate_torvik_teamName(test_teamName, kaggle_teamName)
                continue
                
        if kaggle_teamName.startswith("S "):
            # Looks like Kaggle uses S, while Torvik uses South
            test_teamName = "South" + kaggle_teamName[kaggle_teamName.index("S") + len("S"):]
            # print(f"No match for '{kaggle_teamName}'\t checking if '{test_teamName}' exists")
            if matches_torvik_teamName(test_teamName):
                print(f"Mutating Torvik dataframe team name from '{test_teamName}' -> '{kaggle_teamName}'")
                mutate_torvik_teamName(test_teamName, kaggle_teamName)
                continue

        if kaggle_teamName.startswith("N "):
            # Looks like Kaggle uses S, while Torvik uses South
            test_teamNames = [
                "North" + kaggle_teamName[kaggle_teamName.index("N") + len("N"):],
                "Northern" + kaggle_teamName[kaggle_teamName.index("N") + len("N"):],
            ]
            # print(f"No match for '{kaggle_teamName}'\t checking if '{test_teamName}' exists")
            matched = False
            for test_teamName in test_teamNames:
                if matches_torvik_teamName(test_teamName):
                    print(f"Mutating Torvik dataframe team name from '{test_teamName}' -> '{kaggle_teamName}'")
                    mutate_torvik_teamName(test_teamName, kaggle_teamName)
                    matched = True
                    break

            if matched:
                continue

        if kaggle_teamName.startswith("E "):
            # Looks like Kaggle uses S, while Torvik uses South
            test_teamName = "Eastern" + kaggle_teamName[kaggle_teamName.index("E") + len("E"):]
            # print(f"No match for '{kaggle_teamName}'\t checking if '{test_teamName}' exists")
            if matches_torvik_teamName(test_teamName):
                print(f"Mutating Torvik dataframe team name from '{test_teamName}' -> '{kaggle_teamName}'")
                mutate_torvik_teamName(test_teamName, kaggle_teamName)
                continue

        if kaggle_teamName in kaggleToTorvikTeamNames:
            print(f"Manual override for '{kaggleToTorvikTeamNames[kaggle_teamName]}' -> '{kaggle_teamName}'")
            torvik_team = torvik_teams[torvik_teams == kaggleToTorvikTeamNames[kaggle_teamName]]
            if torvik_team.shape[0] != 1:
                raise Exception(f"Expected matching Torvik team name in dataframe as dictionary is populated with key {kaggle_teamName}")
            mutate_torvik_teamName(kaggleToTorvikTeamNames[kaggle_teamName], kaggle_teamName)
            continue
        
        similar = torvik_teams[torvik_teams.str.startswith(kaggle_teamName[0])]
        raise Exception(f"No team name conversion found for Kaggle team name '{kaggle_teamName}' in year {year}\nThe following are Torvik team names that start with the same character\n{similar}")

    return torvik_df

In [11]:
def get_torvik_data_by_year(year, kaggle_teamNames_round_64):
    m_torvik_team = pd.read_csv(f"./datasets/torvik/mens/team_{year}.csv")
    m_torvik_team = m_torvik_team.dropna(axis=1, how="all")
    
    # Add a winning percentage column
    m_torvik_team["WinPercentage"] = 100.0 * m_torvik_team["Wins"] / m_torvik_team["Games Played"]

     # Add a top 30 wins column
    m_torvik_top30_wins = pd.read_csv(f"./datasets/torvik/mens/top30_wins_{year}.csv")
    m_torvik_top30_wins = m_torvik_top30_wins.dropna(axis=1, how="all")
    m_torvik_top30_wins.rename(columns={"Wins": "Top30Wins"}, inplace=True)
    m_torvik_team = m_torvik_team.merge(m_torvik_top30_wins[["TeamName", "Top30Wins"]], how="left", on="TeamName")
    
    m_torvik_team.fillna({"Top30Wins": 0}, inplace=True) # teams that have zero games played against top 30 do not appear in the top 30 dataset

    return preprocess_torvik_dataframe(year, m_torvik_team, kaggle_teamNames_round_64)

In [17]:
# For each year Y: 
# For each team T that played in the tournament in year Y (ignore teams eliminated in play-in round):
# Record the number of wins by team T in the tournament in year Y, and collect the covariates for OLRE model

# Covariates:
# winning percentage at the end of the regular season (WINPCT), get this from Torvik instead of Kaggle
# point differential at the end of the regular season (DIFF)
# wins above bubble (WAB)
# number of wins against teams rated in the Top 30 based on Torvik's ratings at the end of the regular season (TOP30WIN)

def get_dataframe_by_year(year):
    tourney_seeds = m_ncaa_tourney_seeds[m_ncaa_tourney_seeds["Season"] == year].reset_index(drop=True)
    tourney_compact = m_ncaa_tourney_compact[m_ncaa_tourney_compact["Season"] == year].reset_index(drop=True)
    
    teamIDs_round_64 = []
    teamNames_round_64 = []
    
    for i in range(tourney_seeds.shape[0]):
        teamID = tourney_seeds.at[i, "TeamID"]
        seed = tourney_seeds.at[i, "Seed"]
        if len(seed) > 3:
            # competed in the play-in round
            # check if they made it past this round
            team_tournament_games_won = tourney_compact[lambda df: df["WTeamID"] == teamID]
            if team_tournament_games_won.shape[0] == 0:
                # this means the play-in team won zero games (i.e. didn't make it past play-in round)
                continue
                
        teamIDs_round_64.append(teamID)
        teamNames_round_64.append(m_teams[m_teams["TeamID"] == teamID].iloc[0]["TeamName"])
    
    if len(teamIDs_round_64) != 64:
        raise Exception(f"Expected 64 team IDs, found {len(teamIDs_round_64)}")
    if len(teamNames_round_64) != 64:
        raise Exception(f"Expected 64 team names, found {len(teamNames_round_64)}")
    
    # For each of these 64 teams, compute how many games they won in the tournament (ignoring play-in games)
    n_tournament_games_won = []
    for teamID in teamIDs_round_64:
        # Tournament games won
        team_tournament_games_won = tourney_compact[lambda df: df["WTeamID"] == teamID]
        seed = tourney_seeds[tourney_seeds["TeamID"] == teamID].iloc[0]["Seed"]
        n_games_won = team_tournament_games_won.shape[0] - (0 if len(seed) == 3 else 1)

        if year == 2021 and teamID == teamNameToID["Oregon"]:
            n_games_won += 1 # COVID forfeit by VCU in first round
        
        n_tournament_games_won.append(n_games_won)
    
    n_tournament_games_won = np.array(n_tournament_games_won, dtype=np.uint8)
    if n_tournament_games_won.sum() != 32 + 16 + 8 + 4 + 2 + 1:
        raise Exception(f"Expected total of {32 + 16 + 8 + 4 + 2 + 1} games won across all teams in the tournament, found {n_tournament_games_won.sum()}")
    
    # For each of these 64 teams, compute point differential winning percentage
    regular_season_point_differential = []
    for teamID in teamIDs_round_64:
        w_games = m_ncaa_regular_season[lambda df: (df["Season"] == year) & (df["WTeamID"] == teamID)]
        l_games = m_ncaa_regular_season[lambda df: (df["Season"] == year) & (df["LTeamID"] == teamID)]
        total_point_differential = (w_games["WScore"] - w_games["LScore"]).sum() + (l_games["LScore"] - l_games["WScore"]).sum()
        regular_season_point_differential.append(int(total_point_differential))
    
    m_ncaa_tourney_wins = pd.DataFrame(np.array([teamIDs_round_64, teamNames_round_64, n_tournament_games_won]).T,
                                       columns=["TeamID", "TeamName", "Tournament Wins"])

    torvik = get_torvik_data_by_year(year, teamNames_round_64)
    Xy_ORLE = torvik.merge(m_ncaa_tourney_wins, on="TeamName", how="right")
    
    if Xy_ORLE.shape[0] != 64:
        raise Exception(f"Expected 64 rows in dataframe (one for each team). Found {Xy_ORLE.shape[0]} rows.")
        
    Xy_ORLE = Xy_ORLE.merge(pd.DataFrame(np.array([teamNames_round_64, regular_season_point_differential]).T,
                                         columns=["TeamName", "PointDIFF"],
                                        ).astype({"TeamName": str, "PointDIFF": np.int32}), on="TeamName", how="inner")
    
    Xy_ORLE = Xy_ORLE[["TeamName", "WinPercentage", "PointDIFF", "WAB", "Top30Wins"]]
    Xy_ORLE = m_ncaa_tourney_wins[["TeamName", "Tournament Wins"]].merge(Xy_ORLE, how="inner", on="TeamName")
    
    if Xy_ORLE.shape[0] != 64:
        raise Exception(f"Expected 64 rows in dataframe (one for each team). Found {Xy_ORLE.shape[0]} rows.")

    if Xy_ORLE.isna().sum().sum() != 0:
        raise Exception("Detected NaN in dataframe")

    Xy_ORLE.to_csv(f"./datasets/Xy_ORLE_{year}.csv", index=False)
    return Xy_ORLE

In [20]:
# years = [2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023, 2024] # skip 2020
years = [2018, 2019, 2021, 2022, 2023, 2024] # skip 2020 (add back 2021 later)

for year in years:
    Xy_ORLE = get_dataframe_by_year(year)
    print(Xy_ORLE.sort_values("WinPercentage", ascending=False).head())

Mutating Torvik dataframe team name from 'Wichita St.' -> 'Wichita St'
Mutating Torvik dataframe team name from 'St. Bonaventure' -> 'St Bonaventure'
Mutating Torvik dataframe team name from 'Murray St.' -> 'Murray St'
Manual override for 'Stephen F. Austin' -> 'SF Austin'
Manual override for 'Cal St. Fullerton' -> 'CS Fullerton'
Mutating Torvik dataframe team name from 'Michigan St.' -> 'Michigan St'
Manual override for 'N.C. State' -> 'NC State'
Mutating Torvik dataframe team name from 'New Mexico St.' -> 'New Mexico St'
Manual override for 'Charleston' -> 'Col Charleston'
Mutating Torvik dataframe team name from 'Kansas St.' -> 'Kansas St'
Manual override for 'Loyola Chicago' -> 'Loyola-Chicago'
Mutating Torvik dataframe team name from 'Wright St.' -> 'Wright St'
Mutating Torvik dataframe team name from 'Georgia St.' -> 'Georgia St'
Mutating Torvik dataframe team name from 'Ohio St.' -> 'Ohio St'
Mutating Torvik dataframe team name from 'Florida St.' -> 'Florida St'
Mutating Torvik 

# Plotting

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

In [ ]:
def plot_by_year(year):
    Xy_ORLE = get_dataframe_by_year(year)
    print(Xy_ORLE["PointDIFF"].isna().sum())
    
    fig, ax = plt.subplots()
    ax.hist(Xy_ORLE["Tournament Wins"])
    ax.set_title(f"Distribution of {year} Tournament Wins")
    ax.set_ylabel("Number of Teams")
    ax.set_xlabel("Number of Tournament Wins")
    fig.savefig(f"./images/distribution-{year}-tournament-wins.png")

    fig, ax = plt.subplots()
    ax.hist(Xy_ORLE["WinPercentage"], bins=10)
    ax.set_title(f"Distribution of Win Percentage By Teams in {year} Tournament")
    ax.set_ylabel("Number of Teams")
    ax.set_xlabel("Win %")
    fig.savefig(f"./images/distribution-{year}-win-percentage.png")

    pointDiff_min, pointDiff_max = int(Xy_ORLE["PointDIFF"].min()), int(Xy_ORLE["PointDIFF"].max())
    print(pointDiff_min, pointDiff_max)
    fig, ax = plt.subplots()

    ax.hist(Xy_ORLE["PointDIFF"].to_numpy(), bins=10)
    print(Xy_ORLE["PointDIFF"].to_numpy(dtype=int))
    ax.set_title(f"Distribution of Regular Season Point Differential By Teams in {year} Tournament")
    ax.set_ylabel("Number of Teams")
    ax.set_xlabel("Point Differentials")
    fig.savefig(f"./images/distribution-{year}-point-differentials.png")

    fig, ax = plt.subplots()
    ax.hist(Xy_ORLE["WAB"])
    ax.set_title(f"Distribution of Wins Above Bubble By Teams in {year} Tournament")
    ax.set_ylabel("Number of Teams")
    ax.set_xlabel("Wins Above Bubble")
    fig.savefig(f"./images/distribution-{year}-wins-above-bubble.png")

    fig, ax = plt.subplots()
    ax.hist(Xy_ORLE["Top30Wins"])
    ax.set_title(f"Distribution of Wins Against Top 30 Teams By Teams in {year} Tournament")
    ax.set_ylabel("Number of Teams")
    ax.set_xlabel("Top 30 Wins")
    fig.savefig(f"./images/distribution-{year}-top-30-wins.png")

In [ ]:
plot_by_year(2024);